# Summary of work done - videos

In [2]:
%load_ext autoreload

In [3]:
%autoreload
import os
import numpy as np
import pandas as pd
import time
import pickle
from tqdm.notebook import tqdm
from numpy.lib.format import open_memmap
from scipy.stats import multivariate_normal, entropy, pearsonr
from scripts.data import *
from scripts.preprocessing import *
from scripts.utils import *
from scripts.main import *

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

# Videos

In [4]:
# load video data
videos_data = load_video_data()
videos_data

,Video,Duration(s),NFrames,FPS,Resolution,Width,Height,FramesTimestamps,VideoSaliency,VideoSaliencyNOcb,VideoSalPercentils,VideoSalPercentilsNOcb,Short Name
VideoName,,,,,,,,,,,,,
Diary,Diary_of_a_Wimpy_Kid_Trailer.mp4,117,2817.0,24.0,"(346, 640)",640.0,346.0,"[0.0, 41.708333333333336, 83.41666666666667, 1...",Diary_of_a_Wimpy_Kid_Trailer_saliency_cb.npy,Diary_of_a_Wimpy_Kid_Trailer_saliency.npy,Diary_of_a_Wimpy_Kid_Trailer_saliency_cb_perce...,Diary_of_a_Wimpy_Kid_Trailer_saliency_percenti...,WK
Fractals,Fun_Fractals_v2.mp4,274,8220.0,30.0,"(360, 640)",640.0,360.0,"[0.0, 33.36666666666667, 66.73333333333333, 10...",Fun_Fractals_v2_saliency_cb.npy,Fun_Fractals_v2_saliency.npy,Fun_Fractals_v2_saliency_cb_percentils.npy,Fun_Fractals_v2_saliency_percentils.npy,FF
Present,The Present_Short.mp4,203,4877.0,24.0,"(720, 1280)",1280.0,720.0,"[0.0, 41.666666666666664, 83.33333333333333, 1...",The Present_Short_saliency_cb.npy,The Present_Short_saliency.npy,The Present_Short_saliency_cb_percentils.npy,The Present_Short_saliency_percentils.npy,TP
Despicable,Three_Little_Kittens_Despicable_Me.mp4,170,4266.0,25.0,"(336, 624)",624.0,336.0,"[0.0, 40.0, 80.0, 120.0, 160.0, 200.0, 240.0, ...",Three_Little_Kittens_Despicable_Me_saliency_cb...,Three_Little_Kittens_Despicable_Me_saliency.npy,Three_Little_Kittens_Despicable_Me_saliency_cb...,Three_Little_Kittens_Despicable_Me_saliency_pe...,DM


Observations:

- not all videos have the same resolutions (size and fps)

Preprocessing steps:
- create frame timestamps (duration/fps)
- calculate saliency

## Video Features

# Eye tracking data

In [5]:
# load et data
data_path = './../results/'

df_wk = pd.read_csv(data_path+'WK/results_nss_diary.csv') 
df_ff = pd.read_csv(data_path+'FF/results_nss_fractals.csv')
df_tp = pd.read_csv(data_path+'TP/results_nss_present.csv')

videos_df = {'Diary': df_wk, 'Fractals': df_ff, 'Present': df_tp}

In [6]:
df_wk

,ID,NSS_MEAN,FIX_IN_VID,FLAG,VIDEO_NAME,ET_FILE,NSS,Sex,Age,SWAN,SWAN_HY,SWAN_IN,FIX_ts,FIX_idx
0,NDARGX173AC3,1.367842,377,0,Diary,./data/ETFinalCutSample/NDARGX173AC3/NDARGX173...,0.713887,1,13.652521,0.611111,0.111111,1.111111,286.144,2
1,NDARGX173AC3,1.367842,377,0,Diary,./data/ETFinalCutSample/NDARGX173AC3/NDARGX173...,0.046256,1,13.652521,0.611111,0.111111,1.111111,452.890,4
2,NDARGX173AC3,1.367842,377,0,Diary,./data/ETFinalCutSample/NDARGX173AC3/NDARGX173...,-0.188007,1,13.652521,0.611111,0.111111,1.111111,653.148,6
3,NDARGX173AC3,1.367842,377,0,Diary,./data/ETFinalCutSample/NDARGX173AC3/NDARGX173...,5.137539,1,13.652521,0.611111,0.111111,1.111111,744.893,8
4,NDARGX173AC3,1.367842,377,0,Diary,./data/ETFinalCutSample/NDARGX173AC3/NDARGX173...,-0.146596,1,13.652521,0.611111,0.111111,1.111111,1003.535,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320114,NDARGC559LN8,1.409104,374,0,Diary,./data/ETFinalCutSample/NDARGC559LN8/NDARGC559...,2.689264,1,6.629591,-0.111111,0.111111,-0.333333,115639.816,770
320115,NDARGC559LN8,1.409104,374,0,Diary,./data/ETFinalCutSample/NDARGC559LN8/NDARGC559...,-0.139720,1,6.629591,-0.111111,0.111111,-0.333333,115839.727,772
320116,NDARGC559LN8,1.409104,374,0,Diary,./data/ETFinalCutSample/NDARGC559LN8/NDARGC559...,-0.241721,1,6.629591,-0.111111,0.111111,-0.333333,116022.988,774
320117,NDARGC559LN8,1.409104,374,0,Diary,./data/ETFinalCutSample/NDARGC559LN8/NDARGC559...,0.948535,1,6.629591,-0.111111,0.111111,-0.333333,116272.886,776
